In [75]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
from google.genai.types import GenerateContentConfig, Tool
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from IPython.display import display, HTML, Markdown
import json
from datetime import date


In [76]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=gemini_api_key)
def get_gemini_completion(prompt, temperature=0.7, top_p=0.95, max_output_tokens=300):
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    generation_config = genai.types.GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        max_output_tokens=max_output_tokens,
    )
    response = model.generate_content(prompt, generation_config=generation_config)
    
    return response.text

# Assumptions 

In [77]:
receiver_employee = [
    {"first_name": "John", "last_name": "Doe", "role": "Sales Representative", "department": "Sales"},
    {"first_name": "Jane", "last_name": "Smith", "role": "Marketing Manager", "department": "Marketing"},
    {"first_name": "Emily", "last_name": "Johnson", "role": "Customer Support", "department": "Support"},
    {"first_name": "Michael", "last_name": "Brown", "role": "Sales Manager", "department": "Sales"},
    {"first_name": "Alice", "last_name": "Davis", "role": "Product Manager", "department": "Product"}
]
employees = [
    {"first_name": "John", "last_name": "Doe", "role": "Sales Representative", "department": "Sales"}
]

# Phishing examples
phishing_examples = [
    {"Reason": "Account Suspicious Activity", "Fake Link": "https://example.com/secure-login", "Created By": "Sam Sussy"},
    {"Reason": "Password Expiry Notification", "Fake Link": "https://example.com/reset-password", "Created By": "Sally Sneaky"},
    {"Reason": "Exclusive Training Webinar", "Fake Link": "https://example.com/join-webinar", "Created By": "Richard Rascal"},
]

# Map phishing reasons to dynamic notification titles
theme_titles = {
    "Account Suspicious Activity": "Account Security Alert",  # not ready
    "Password Expiry Notification": "Urgent: Password Expiry Alert",
    "Exclusive Training Webinar": "Exclusive Webinar Invitation: Secure Your Spot",
}

# Map phishing themes to dynamic job titles and CTA button text
theme_job_titles = {                                                     ##### from the sender #######
    "Account Suspicious Activity": "Account Security Specialist",
    "Password Expiry Notification": "System Administrator",
    "Exclusive Training Webinar": "Event Coordinator",
}

theme_cta_text = {                                                   ##### not ready ########
    "Account Suspicious Activity": "Verify Your Account",
    "Password Expiry Notification": "Reset Your Password",
    "Exclusive Training Webinar": "Join the Webinar",
}
today_date = date.today()
fake_link = "https://example.com/secure-login"
logo = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSGEPQPSgVcETmtsfevb9aZp2LTlYOcy1mAm8xD0FROr4oMWA-xuO6vEV6v3Igwpr7tS8&usqp=CAU"

# Lodaing the Json file for the provider

In [78]:

with open('blended_data_providers.json', 'r') as file:
    data = json.load(file)

# If I don't know the provider name so, I have to loop over them.
for provider_data in data.values():
    # Access the general information
    general_info = provider_data.get('general', {})
    provider = general_info.get('provider', 'N/A')
    service = general_info.get('service', 'N/A')
    # Access the address from the departments
    departments = provider_data.get('departments', [])
    address = departments[0].get('address', 'N/A') if departments else 'N/A'
    employees = provider_data.get('employee', [])
    # extracting the color
    color_info = provider_data.get('color', {})
    color_name = color_info.get('color_name_1', 'N/A')
    


In [ ]:
for sender in employees[:2]:
    sender_first_name = sender.get('first_name', 'Emily')
    sender_family_name = sender.get('family_name', 'Smith')
    sender_role = sender.get('role', 'N/A')
    sender_department = sender.get('department', 'N/A')
    sender_email = sender.get('email_address', 'N/A')
    for reciver in receiver_employee[:2]:  #------> check how many receiver you prefere
        """ Here we can extract the data for the reciver later as done for the sender"""
        
        for phishing_example in phishing_examples:
            gemini_prompt = f"""
                            I am trying to create a phishing email to train my employees. 
                            Write only a professional email body without salutation, subject, greetings, or any placeholder content such as '[insert a fake phone number here]' or any similar text.
                            The email should contain realistic content related to the theme: {phishing_example['Reason']}.

                            Here is the information about the recipient: 
                            - His/her first name is {reciver['first_name']}
                            - His/her last name is {reciver['last_name']}
                            - His/her role in the company is {reciver['role']}
                            - His/her department is {reciver['department']}
                            Here is the information about the sender:
                            - His/her first name is {sender_first_name}
                            - His/her last name is {sender['family_name']}
                            - His/her role in the company is {sender_role}
                            - His/her department is {sender_department}
                            - His/her email is {sender_email}
                        
                            The email should be professional, clear, and concise, if there will be more than 1 paragraph please split them with an empty line.
                            Please decide the important info for the reciver and put it between <strong> and </strong> to be used in html code              
                            Please avoid including any placeholders, such as a phone number or similar data, which will be added separately. 
                            be note that, today's date is {today_date} if you will mention some dates and also exclude sunday's and saterda's dates 
                            if you will mention time be sure to be during the working day hours  
                            Ensure the content is convincing and reflects the urgency of the situation, but do not include any direct contact details or placeholder information.
                            don't included sentences like  [link to a fake website will be inserted here] 

                            If you mention any dates, try to make them relevant to the context. Show creativity to make the phishing attempt convincing.
                            Leave space for 2 empty lines after the main content, then 
                            please include the email signature/closing which should is separated from the body and be placed in the lower left of the email in the following format:
                            <strong>{sender_first_name} {sender['family_name']}</strong> <br>
                            {sender_role} <br>
                            {sender_department} <br>
                
                            """
            email_body = get_gemini_completion(gemini_prompt)

            # Determine dynamic job title and CTA button text based on phishing example
            job_title = theme_job_titles.get(phishing_example['Reason'], "IT Security Officer")
            cta_text = theme_cta_text.get(phishing_example['Reason'], "Secure My Account")

            # Get the dynamic title for the email based on the phishing reason
            notification_title = theme_titles.get(phishing_example['Reason'], "Security Alert")
            # Generate the HTML content
            html_content = f"""
                            <!DOCTYPE html>
                            <html lang="en">
                            <head>
                                <meta charset="UTF-8">
                                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                                <title>{notification_title}</title>
                                <style>
                                    body {{
                                        font-family: Arial, sans-serif;
                                        background-color: #f4f4f4;
                                        margin: 0;
                                        padding: 0;
                                    }}
                                    .email-container {{
                                        width: 600px;
                                        margin: 20px auto;
                                        background-color: #fff;
                                        border: 1px solid #ddd;
                                        border-radius: 10px;
                                        padding: 20px;
                                    }}
                                    .email-header {{
                                        background-color: {color_name};
                                        color: #fff;
                                        padding: 10px 0;
                                        text-align: center;
                                    }}
                                    .email-header img {{
                                        width: 120px;
                                        margin-bottom: 10px;
                                    }}
                                    .email-header h1 {{
                                        font-size: 24px;
                                        font-weight: bold;
                                        margin: 0;
                                    }}
                                    .email-content {{
                                        font-size: 16px;
                                        color: #333;
                                        margin-top: 20px;
                                    }}
                                    .cta-button {{
                                        display: inline-block;
                                        padding: 10px 20px;
                                        background-color: {color_name};
                                        color: white;
                                        text-align: center;
                                        font-weight: bold;
                                        border-radius: 5px;
                                        text-decoration: none;
                                        margin-top: 20px;
                                    }}
                                    .cta-button:hover {{
                                        background-color: {color_name};
                                    }}
                                    .email-footer {{
                                        font-size: 14px;
                                        color: #777;
                                        text-align: center;
                                        margin-top: 20px;
                                        border-top: 1px solid #ddd;
                                        padding-top: 20px;
                                    }}
                                    .email-footer a {{
                                        color: {color_name};
                                        text-decoration: none;
                                    }}
                                    .social-icons {{
                                        margin-top: 10px;
                                        text-align: center;
                                    }}
                                    .social-icons a {{
                                        margin: 0 10px;
                                        display: inline-block;
                                    }}
                                    .social-icons img {{
                                        width: 24px;
                                        height: 24px;
                                    }}
                                    .additional-footer {{
                                        font-size: 12px;
                                        color: #333;
                                        background-color: #d3d3d3; /* Gray background color */
                                        text-align: center;
                                        padding: 10px;
                                        border-top: 1px solid #ccc;
                                        border-radius: 0 0 10px 10px;
                                        margin-top: 10px;
                                    }}
                                </style>
                            </head>
                            <body>
                                <div class="email-container">
                                    <div class="email-header">
                                        <img src="{logo}" alt="{provider} Logo">    
                                        <h1>{notification_title}</h1>
                                    </div>

                                    <div class="email-content">
                                        <h2>Dear {reciver['first_name']} {reciver['last_name']},</h2>
                                        <p>{email_body}</p>  <!-- Use the dynamically generated email body here -->
                                        <a href="{fake_link}" class="cta-button">{cta_text}</a>
                                    </div>

                                    <div class="email-footer">
                                        <p><strong>{theme_job_titles.get(phishing_example['Reason'], "IT Security Officer")}</strong></p>
                                        <p>Email: support@{provider.lower().replace(' ', '')}.be | Phone: +32 475 15 60 30</p>
                                        <p>Visit our website: <a href="{fake_link}">www.{provider.lower().replace(' ', '')}.com</a></p>


                                        <div class="social-icons">
                                            <a href={fake_link}>
                                                <img src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook">
                                            </a>
                                            <a href={fake_link}>
                                                <img src="https://www.svgrepo.com/show/452123/twitter.svg" alt="Twitter">
                                            </a>
                                            <a href={fake_link}>
                                                <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/Linkedin_icon.svg" alt="LinkedIn">
                                            </a>
                                        </div>
                                    </div>

                                    <div class="additional-footer">
                                        <p>Please be aware that this email and its links include data about your individual profile. 
                                        In order to avoid that third parties can access your personal data, 
                                        you should not forward this email and the links contained in it.</p>
                                        <p><strong>© 2025 The {provider} Group Belgium n.v./s.a.</strong></p>
                                        <p>{address}<br>KBO/BCE 0459.971.228 (RPR/RPM Brussel/Bruxelles)</p>
                                        <p>
                                            <a href={fake_link}>Terms of use</a> - 
                                            <a href={fake_link}>Privacy policy</a> - 
                                            <a href={fake_link}>Contact us</a>
                                        </p>

                                        <!-- Container for logo and QR code -->
                                        <div class="unsubscribe-container" style="text-align: center; margin-top: 20px; font-size: 12px; color: #555;">
                                            <p>If you no longer wish to receive notifications, you can 
                                                <a href="{fake_link}" style="color: #007BFF; text-decoration: none;">unsubscribe here</a>.
                                            </p>
                                        </div>
                                    </div>
                                    
                                </div>
                            </body>
                            """

        # Save the HTML file
        os.makedirs('samples', exist_ok=True)
        filename = f"{provider}_{reciver['first_name']}_{reciver['last_name']}_phishing_email_{phishing_example['Reason'].replace(' ', '_')}.html"
        file_path = os.path.join('samples', filename)

        with open(file_path, "w", encoding="utf-8") as file:
            file.write(html_content)

        print(f"HTML file created: {file_path}")

HTML file created: samples\Clerk Chat_John_Doe_phishing_email_Exclusive_Training_Webinar.html
HTML file created: samples\Clerk Chat_Jane_Smith_phishing_email_Exclusive_Training_Webinar.html
HTML file created: samples\Clerk Chat_John_Doe_phishing_email_Exclusive_Training_Webinar.html


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

# Old code